In [1]:
import torch
import seaborn as sns
import sys
from pathlib import Path
sys.path.insert(0, "../..")
from src.data import make_dataset
from src.models import imagemodels
from src.models import train_model
from src.models import metrics
import gin

/home/azureuser/.cache/pypoetry/virtualenvs/poetry_add_pandas-Tn0MmM9i-py3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-31 09:34:43.644217: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-31 09:34:46.127226: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-31 09:34:46.127260: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore abov

In [2]:
gin.parse_config_file("model.gin")
train_dataloader, test_dataloader = make_dataset.get_MNIST()
accuracy = metrics.Accuracy()
# print(gin.config_str())

In [3]:

def hand_hypertuning(
    learning_rates,
    kernel_sizes,
    filters1,
    filters2,
    units1,
    units2,
    model_dic,
    epoch = 3,
    num_classes = 10,
    log_dir = "../../models/test/"):
  
    # make list if not list:
    def make_ls(x):
        return x if type(x) == type([]) else [x]

    # maak van loopers list if not:
    learning_rates = make_ls(learning_rates) 
    kernel_sizes = make_ls(kernel_sizes) 
    filters1 = make_ls(filters1) 
    filters2 = make_ls(filters2) 
    units1 = make_ls(units1) 
    units2 = make_ls(units2)

    #model naam en model object vullen:
    model_name = model_dic[0]
    
    
    #gin_config resetten en vullen:
    gin.clear_config()
    gin.parse_config_file("model.gin")
    
    gin.bind_parameter("trainloop.log_dir",log_dir)
    gin.bind_parameter("trainloop.epochs", epoch)
    gin.bind_parameter(model_name + '.num_classes', num_classes)
    # gin.bind_parameter(model_name + '.kernel_size', kernel_size)

    for fl1 in filters1:
        for fl2 in filters2:
            if fl2 <= fl1:
                for ut1 in units1:
                    for ut2 in units2:
                        if ut2 <= ut1:
                            for lr in learning_rates:
                                for kerz in kernel_sizes:
                                    
                                    # pm_dic = {
                                    #     fl1 : model_name + '.filter1'
                                    #     ,fl2 : model_name + '.filter2'
                                    #     ,ut1 : model_name + '.unit1'
                                    #     ,ut2 : model_name + '.unit2'
                                    #     ,kerz : model_name + '.kernel_size'
                                    #     ,lr : 'trainloop.learning_rate'
                                    # }
                                    # print(pm_dic)
                                    # for pm in pm_dic:
                                    #     print(pm)
                                    #     print(pm_dic[pm])
                                    #     gin.bind_parameter(pm_dic[pm],pm)


                                
                                    gin.bind_parameter(model_name + '.filter1', fl1)
                                    gin.bind_parameter(model_name + '.filter2', fl2)
                                    gin.bind_parameter(model_name + '.unit1', ut1)
                                    gin.bind_parameter(model_name + '.unit2', ut2)
                                    gin.bind_parameter(model_name + '.kernel_size', kerz)
                                    gin.bind_parameter("trainloop.learning_rate", lr)
                                    # print(gin.config_str())

                                    model = model_dic[1]

                                    model =  train_model.trainloop(
                                        model=model(),
                                        metrics=[accuracy],
                                        train_dataloader=train_dataloader,
                                        test_dataloader=test_dataloader,
                                        train_steps=len(train_dataloader),
                                        eval_steps=150,
                                    )
    
    return None

model_dic = {
    'CNN':{0:'CNN',1:imagemodels.CNN}
    ,'CNN_150': {0:'CNN_150',1:imagemodels.CNN_150}
    }
    


Als eerste wordt de learning rate getest:

In [5]:

hand_hypertuning(
    model_dic = model_dic['CNN']
    ,learning_rates = [0.01, 0.001, 0.0001]
    ,kernel_sizes = 3
    ,filters1 = 128
    ,filters2 = 128
    ,units1 = 128
    ,units2 = 64
    ,epoch = 10
    ,log_dir = "../../models/learning_rate/"
    )

2022-12-29 12:45:15.170 | INFO     | src.data.data_tools:dir_add_timestamp:114 - Logging to ../../models/test/20221229-1245
  0%|          | 0/3 [00:15<?, ?it/s]


KeyboardInterrupt: 

Nu we weten dat de optimale learning rate ligt op ongeveer 0.001 kan verder naar andere optimuns worden gezocht.
Als eerste wordt kernelsize getest:

Kernal; size is optimaal op 2 omdat filters en units de onformatie aan elkaar doorgeven, hebben filter zise en unitr een invloed op elkaar. De units nemen het over op een platten 128. dit geeft beperkingen deze zijn meegenomen:

In [ ]:
fil_range_e3 = [16,32,64,128]
fil_range_e10 = [32,64]

hand_hypertuning(
    model_dic = model_dic['CNN']
    ,learning_rates = 0.001
    ,kernel_sizes = 3
    ,filters1 = fil_range_e10
    ,filters2 = fil_range_e10
    ,units1 = 128
    ,units2 = 64
    ,log_dir = "../../models/filters/"
    ,epoch = 10
    )

In [ ]:
unit_range_e3 = [16,32,64,128]
unit_range_e10 = [32,64,128]

hand_hypertuning(
    model_dic = model_dic['CNN']
    ,learning_rates = 0.001
    ,kernel_sizes = 3
    ,filters1 = 64
    ,filters2 = 64
    ,units1 = 128
    ,units2 = 128
    ,log_dir = "../../models/units/"
    ,epoch = 10
    )

In [4]:
fil_range_e3 = [16,32,64,128]
fil_range_e10 = [32,64]

hand_hypertuning(
    model_dic = model_dic['CNN_150']
    ,learning_rates = 0.001
    ,kernel_sizes = 2
    ,filters1 = fil_range_e10
    ,filters2 = fil_range_e10
    ,units1 = 128
    ,units2 = 128
    ,log_dir = "../../models/filters_cnn150/"
    ,epoch = 10
    )

2022-12-31 09:34:58.413 | INFO     | src.data.data_tools:dir_add_timestamp:114 - Logging to ../../models/filters_cnn150/20221231-0934
100%|██████████| 1875/1875 [00:56<00:00, 33.39it/s]
2022-12-31 09:35:57.622 | INFO     | src.models.train_model:trainloop:171 - Epoch 0 train 0.6369 test 0.4305 metric ['0.8471']
100%|██████████| 1875/1875 [00:54<00:00, 34.56it/s]
2022-12-31 09:36:53.940 | INFO     | src.models.train_model:trainloop:171 - Epoch 1 train 0.3831 test 0.3645 metric ['0.8610']
100%|██████████| 1875/1875 [00:55<00:00, 33.93it/s]
2022-12-31 09:37:51.348 | INFO     | src.models.train_model:trainloop:171 - Epoch 2 train 0.3261 test 0.3377 metric ['0.8740']
100%|██████████| 1875/1875 [00:54<00:00, 34.20it/s]
2022-12-31 09:38:48.257 | INFO     | src.models.train_model:trainloop:171 - Epoch 3 train 0.2870 test 0.3483 metric ['0.8712']
100%|██████████| 1875/1875 [00:55<00:00, 33.86it/s]
2022-12-31 09:39:45.643 | INFO     | src.models.train_model:trainloop:171 - Epoch 4 train 0.2686 t

KeyboardInterrupt: 

In [ ]:
%tensorboard